# Maryland Brewing Web Scraping

In [2]:
#import all required libraries
from configparser import ConfigParser
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
from datetime import date
import pandas as pd
import re



#Add User agent to scrape the site

service = ChromeService(executable_path=ChromeDriverManager().install())
options = ChromeOptions()


options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36")


### Jailbreak Brewing

In [3]:
driver = webdriver.Chrome(options=options, service=service)

driver.get('https://jailbreakbrewing.com')
button = driver.find_element(By.TAG_NAME, 'button')
button.click()
time.sleep(10)
jail_cols = driver.find_elements(By.CLASS_NAME,'whatsontaphdr')
jailbreak = pd.DataFrame(columns = ['Brewery','Date_Retrieved']+[col.text for col in jail_cols])
jail_beers = driver.find_elements(By.CLASS_NAME,'whatsontap')
i = 0
for col in jailbreak.columns[2:]:
    jailbreak[col] = [j.text for j in jail_beers[i::4]]
    i +=1
jailbreak['Brewery'] = 'Jailbreak Brewing'
jailbreak['Date_Retrieved'] = f"{date.today().month}/{date.today().day}/{date.today().year}"

In [46]:
driver.quit()

### Black Flag Brewing Co

In [5]:
driver.get("https://www.blackflagbrewingco.com")

In [6]:
button = driver.find_element(By.ID,"ageYes")
button.click()

In [8]:
blackflag

,Brewery,Date_Retrieved,Name,Type,Profile,ABV%,IBU


In [9]:
elements = driver.find_element(By.CLASS_NAME,'col-md-12')
flag_beers = elements.find_elements(By.TAG_NAME,'td')

i = 0
for col in blackflag.columns[2:]:
    blackflag[col] = [j.text for j in flag_beers[i::5]]
    i +=1
blackflag['Brewery'] = 'Black Flag Brewing'
blackflag['Date_Retrieved'] = f"{date.today().month}/{date.today().day}/{date.today().year}"



In [122]:
blackflag.rename(columns = {'Type':'Style',
                           'ABV%':'ABV'},inplace = True)

In [53]:
driver.quit()

In [54]:
driver = webdriver.Chrome(options=options, service=service)

# driver.get("https://elderpine.com")
# time.sleep(5)
# driver.find_element(By.ID,"yui_3_17_2_1_1665263650447_375").click()
driver.get("https://www.elderpine.com/tasting-room")

In [24]:
import re

In [25]:
dir(re)

['A',
 'ASCII',
 'DEBUG',
 'DOTALL',
 'I',
 'IGNORECASE',
 'L',
 'LOCALE',
 'M',
 'MULTILINE',
 'Match',
 'Pattern',
 'RegexFlag',
 'S',
 'Scanner',
 'T',
 'TEMPLATE',
 'U',
 'UNICODE',
 'VERBOSE',
 'X',
 '_MAXCACHE',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__version__',
 '_cache',
 '_compile',
 '_compile_repl',
 '_expand',
 '_locale',
 '_pickle',
 '_special_chars_map',
 '_subx',
 'compile',
 'copyreg',
 'enum',
 'error',
 'escape',
 'findall',
 'finditer',
 'fullmatch',
 'functools',
 'match',
 'purge',
 'search',
 'split',
 'sre_compile',
 'sre_parse',
 'sub',
 'subn',
 'template']

In [ ]:
re.findall()

In [138]:
driver.get("https://www.elderpine.com/tasting-room")

In [55]:
for e in driver.find_elements(By.CLASS_NAME,'row sqs-row'):
    print(e.text)

In [90]:
ep_list = [e.text for e in driver.find_elements(By.CSS_SELECTOR,'.row p')]

In [94]:
for ind, ele in enumerate(ep_list):
    if ele.startswith('1.'):
        start_list = ind
        break
        
for ind, ele in enumerate(ep_list):
    if ele.startswith('HOME'):
        end_list = ind
        break

In [95]:
ep_list = ep_list[start_list:end_list]

In [101]:
ep_list = [x for x in ep_list if x != 'Inside Bar Only']
name = ep_list[1::4]
style = ep_list[2::4]
abv = ep_list[3::4]

In [113]:
[4]*4

[4, 4, 4, 4]

In [125]:
elderpine = pd.DataFrame({'Brewery':['Elder Pine Brewing']*len(style),
                          'Date_Retrieved':[f"{date.today().month}/"+
                                            f"{date.today().day}/{date.today().year}"]*len(style),
                          'Name':name,
                          'Style':style,
                          'ABV': abv})

In [126]:
elderpine = elderpine.loc[~elderpine.Name.str.startswith('Coming Soon')]

In [127]:
elderpine.ABV = elderpine.ABV.str.replace(' ABV','')

In [128]:
elderpine.head()

,Brewery,Date_Retrieved,Name,Style,ABV
0,Elder Pine Brewing,10/8/2022,Cerveza Viejo Especial,Mexican-style Lager Blend,4.9%
1,Elder Pine Brewing,10/8/2022,Ameno,Italian-style Pilsner,5%
2,Elder Pine Brewing,10/8/2022,Nordic Brume,Hazy IPA,7%
3,Elder Pine Brewing,10/8/2022,Barrel-aged Tmavé Pivo,Czech-style Dark Lager,6.6%
4,Elder Pine Brewing,10/8/2022,Oryza Villein,Dry-Hopped Rice Saison,6%


In [129]:
df = pd.concat([jailbreak,elderpine,blackflag],axis = 0,ignore_index = True)

In [131]:
df['ABV'] = df.ABV.str.replace('%','')
df['ABV'] = df.ABV.astype(float)

In [134]:
df.shape

(49, 7)

In [137]:
df.sort_values(by = ['Style','ABV'])

,Brewery,Date_Retrieved,Name,Style,ABV,IBU,Profile
1,Jailbreak Brewing,10/8/2022,The Infinite,Amber Ale,5.0,19,NaN
4,Jailbreak Brewing,10/8/2022,The Lesser Jesus,American Pale Ale,4.8,30,NaN
11,Jailbreak Brewing,10/8/2022,Fall Funk,Apple Cinnamon Berliner Weisse,4.0,N/A,NaN
22,Elder Pine Brewing,10/8/2022,Double Oaked The Sun’s Gone Dim And The Sky’s ...,Barrel-aged Baltic Porter,10.0,NaN,NaN
31,Elder Pine Brewing,10/8/2022,Summon the Moon Lord,Barrel-aged English Barleywine,16.0,NaN,NaN
28,Elder Pine Brewing,10/8/2022,The Last Lights Are All Fire,Barrel-aged Fruited Sour,7.8,NaN,NaN
30,Elder Pine Brewing,10/8/2022,Horizons Shifting Slow in Blue,Barrel-aged Fruited Sour,7.8,NaN,NaN
3,Jailbreak Brewing,10/8/2022,Belgo Bleek,Belgian Pale Ale,5.5,N/A,NaN
12,Jailbreak Brewing,10/8/2022,Vinous Funk *Can Pour*,Black Currant Kettle-Soured Berliner Weisse,4.0,N/A,NaN
6,Jailbreak Brewing,10/8/2022,Dark & Righteous,Black IPA,6.5,N/A,NaN


In [136]:
driver.quit()